## Deconvolution Test


In [1]:
from s2cnn import S2Convolution

import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import spatial

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from average_meter import AverageMeter
from data_source import DataSource
from data_splitter import DataSplitter
from database_parser import DatabaseParser

from loss import ImprovedTripletLoss
from mission_indices import MissionIndices
from model import Model
from sphere import Sphere
from training_set import TrainingSet
from visualize import Visualize
from model_encode_decode_simple import ModelEncodeDecodeSimple

from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from s2cnn import so3_near_identity_grid, S2Convolution, s2_near_identity_grid, SO3Convolution, so3_integrate

from tqdm.auto import tqdm

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
#torch.cuda.set_device(0)
#torch.backends.cudnn.benchmark = True

bandwidth = 30
restore = False
n_epochs = 55
batch_size = 13
num_workers = 32
descriptor_size = 256
net_input_size = 2 * bandwidth
n_features = 3
cache = 1
n_data = 52

#net = ModelEncodeDecodeSimple(bandwidth).cuda()
net = ModelEncodeDecodeSimple(bandwidth)
v = Visualize()

In [3]:
dataset_path = "/mnt/data/datasets/Spherical/test_training_data/"
db_parser = DatabaseParser(dataset_path)

ds = DataSource(dataset_path, cache)
ds.load(n_data)

Reading missions db from /mnt/data/datasets/Spherical/test_training_data/missions.csv
Read    mission_anchor mission_positive mission_negative
0      6725..0000       6725..0000       6725..0000
1      6725..0000       6725..0000       6725..0000
2      4e53..0000       7799..0000       4e53..0000
3      4e53..0000       7799..0000       4e53..0000
4      4e53..0000       7799..0000       4e53..0000
5      4e53..0000       7799..0000       4e53..0000
6      4e53..0000       7799..0000       4e53..0000
7      4e53..0000       7799..0000       4e53..0000
8      4e53..0000       7799..0000       4e53..0000
9      4e53..0000       7799..0000       4e53..0000
10     4e53..0000       7799..0000       4e53..0000
11     4e53..0000       7799..0000       4e53..0000
12     4e53..0000       7799..0000       4e53..0000
13     4e53..0000       7799..0000       4e53..0000
14     4e53..0000       7799..0000       4e53..0000
15     4e53..0000       7799..0000       4e53..0000
16     4e53..0000       7


Loading positives from:	/mnt/data/datasets/Spherical/test_training_data//training_positive_pointclouds/ and /mnt/data/datasets/Spherical/test_training_data//training_positive_sph_images/



Loading negatives from:	/mnt/data/datasets/Spherical/test_training_data//training_negative_pointclouds/ and /mnt/data/datasets/Spherical/test_training_data//training_negative_sph_images/



Done loading dataset.
	Anchor point clouds total: 	52
	Anchor images total: 		52
	Anchor poses total: 		52
	Positive point clouds total: 	52
	Positive images total: 		52
	Positive poses total: 		52
	Negative point clouds total: 	52
	Negative images total: 		52
	Negative poses total: 		52


In [4]:
train_set = TrainingSet(restore, bandwidth)
train_set.generateAll(ds)

Generating anchor spheres



Generating positive spheres



Generating negative spheres



Generated features
a features 370044, p features 371550, n features 368757,
Generating features from 0 to 52


In [5]:
print("Total size: ", len(train_set))
split = DataSplitter(train_set, restore, test_train_split=1.0, shuffle=True)

train_loader, val_loader, test_loader = split.get_split(
    batch_size=batch_size, num_workers=num_workers)
train_size = split.get_train_size()
val_size = split.get_val_size()
test_size = split.get_test_size()
print("Training size: ", train_size)
print("Validation size: ", val_size)
if test_size == 0:
    print('Test size is 0. Configured for external tests')
else:
    print("Testing size: ", test_size)

Total size:  52
Training size:  46
Validation size:  6
Test size is 0. Configured for external tests


## Debug Visualization

In [6]:
# For the datasource 
(a, p, n) = ds.get_all_cached_clouds()
first_anchor = a[0]
print(f"Anchor shape is {first_anchor.shape}")
v.visualizeRawPointCloud(first_anchor, jupyter=False)

Anchor shape is (65536, 4)


In [8]:
# For the train set
anchor_features = train_set.anchor_features
print(f"Anchor feature length is {len(anchor_features)}")
anchor_feat, _, _ = train_set.__getitem__(0)
print(f"Anchor feature shape is {anchor_feat.shape}")
anchor_feat = anchor_features[0]
print(f"Anchor feature shape is {anchor_feat.shape}")
print(f"Non zero features: {np.count_nonzero(anchor_feat)}")
sphere_anchor = Sphere(bw=bandwidth, features=anchor_feat)
v.visualizeSphere(sphere_anchor)

Anchor feature length is 52
Anchor feature shape is torch.Size([2, 60, 60])
Anchor feature shape is (2, 60, 60)
Non zero features: 7200


In [6]:
net.eval()                                                                  
with torch.no_grad():
    for batch_idx, (data1, data2, data3) in enumerate(train_loader):
        #data1, data2, data3 = data1.cuda().float(), data2.cuda().float(), data3.cuda().float()      
        data1, data2, data3 = data1.float(), data2.float(), data3.float()
        break

In [15]:
x_init = torch.clone(data1)
print(f"input shape: {data1.shape}")
x_enc_dec = net(data1)
print(f"computed embedding: {x_enc_dec.shape}")

input shape: torch.Size([13, 2, 60, 60])
[conv] x shape is torch.Size([900, 13, 2, 2]) and y shape is torch.Size([900, 2, 10, 2])
encoded x shape is torch.Size([13, 10, 60, 60, 60])
integrated x shape is torch.Size([13, 10, 60, 60])
[deconv] x shape is torch.Size([900, 13, 10, 2]) and y shape is torch.Size([900, 10, 2, 2])
z shape is torch.Size([10, 2])


RuntimeError: shape '[1, 1, 1, 2, 2]' is invalid for input of size 40

In [16]:
features_before = x_init[0,:,:,:].detach().numpy()
features_after = x_enc_dec[0,:,:,:].detach().numpy()

print(f"features before {features_before.shape}, features after {features_after.shape}")
sphere_before = Sphere(bw=bandwidth, features=features_before)
sphere_after = Sphere(bw=bandwidth, features=features_after)

v.visualizeSphere(sphere_before, jupyter=False)
#v.visualizeSphere(sphere_after, jupyter=False)

features before (2, 60, 60), features after (2, 60, 60)


In [17]:
features_diff_range = features_before[0,:,:]-features_after[0,:,:]
features_diff_intensity = features_before[1,:,:]-features_after[1,:,:]

range_l2 = np.linalg.norm(features_diff_range, ord=2)
intensity_l2 = np.linalg.norm(features_diff_intensity, ord=2)

print(f"Range L2 is {range_l2} and Intensity L2 is {intensity_l2}")

Range L2 is 1237.4351806640625 and Intensity L2 is 20686.00390625
